In [6]:
from src.helper import *
from src.CONSTANTS import *
import matplotlib.pyplot as plt
import wget
import pandas as pd
import numpy as np
import h5py as h5
from random import shuffle
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Input, Dense, GRU, Embedding, Multiply, Attention, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras import backend as k


# Download the dataset
url = 'https://figshare.com/ndownloader/files/12506534'
#wget.download(url)

# Read the downloaded data to a dataframe
with h5.File('D:/MS2/src/holdout_hcd.hdf5', 'r') as f:
  KEY_ARRAY = ["sequence_integer", "precursor_charge_onehot", "intensities_raw"]
  KEY_SCALAR = ["collision_energy_aligned_normed", "collision_energy"]
  df = pd.DataFrame({key: list(f[key][...]) for key in KEY_ARRAY})
  for key in KEY_SCALAR:
    df[key] = f[key][...]

# Add convenience columns
df['precursor_charge'] = df.precursor_charge_onehot.map(lambda a: a.argmax() + 1)
df['sequence_maxquant'] = df.sequence_integer.map(lambda s: "".join(PROSIT_INDEXED_ALPHABET[i] for i in s if i != 0))
df['sequence_length'] = df.sequence_integer.map(lambda s: np.count_nonzero(s))

In [7]:
from pyteomics import mass

df['mass'] = df.sequence_maxquant.map(lambda b: mass.calculate_mass(b))

print(df.columns)

PyteomicsError: Pyteomics error, message: 'Could not create a Composition object from string: "ILM(ox)KSVR": not a valid sequence or formula'